In [1]:
# Import libaries

from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [2]:
final_list = []

# Skytrax URL
url = 'https://www.airlinequality.com/airline-reviews/jetstar-airways/?sortby=post_date%3ADesc&pagesize=100'

# Request URL
response = requests.get(url)

# Create Soup
soup = BeautifulSoup(response.content, 'lxml')

# For loop for HTML
for idx, review in enumerate(soup.find_all('div',{'class':"review-stats"})):
    
    # Create a empty dictionary to contain columns and values
    review_dict = dict()
    
    # Values that is in star fill
    names = review.find_all('td', class_='review-rating-header')[::-1]
    ratings = review.find_all('td', class_='stars')[::-1]
    
    for name, rating in zip(names[1:], ratings):
        review_dict[name.text] = rating.find_all(class_='fill')[-1].text
    
    # Author, Date, Overall_rating,Text_content, Recommended
    review_dict['Author'] = soup.find_all(itemprop='name')[idx+1].text
    review_dict['Date'] = soup.find_all(itemprop='datePublished')[::2][idx].attrs['content']
    review_dict['Overall_rating'] = soup.find_all('div',{'itemprop':'reviewRating'})[idx].find(itemprop="ratingValue").text
    review_dict['text_content'] = soup.find_all('div',class_='text_content')[idx].text
    review_dict['Recommended'] = soup.find_all('div',{'class':"review-stats"})[idx].find_all('tr')[-1].find_all('td')[-1].text
        
    # Type of Traveller
    try:
        review.find('td',{'class':'review-rating-header type_of_traveller'}).text is not None
        review_dict['traveller_type'] = soup.find_all('td',{'class':'review-rating-header type_of_traveller'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['traveller_type'] = np.nan
    
    # Seat type
    try:
        review.find('td',{'class':'review-rating-header cabin_flown'}).text is not None
        review_dict['seat_type'] = soup.find_all('td',{'class':'review-rating-header cabin_flown'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['seat_type'] = np.nan
    
    
    # Append dictionary into a list
    final_list.append(review_dict)

# Create DataFrame
df = pd.DataFrame(final_list)
df['airline'] = 'jetstar airways'

# rearrange dataframe
df = df[['airline','Author','Date','text_content','traveller_type','seat_type','Overall_rating','Seat Comfort',
 'Cabin Staff Service','Food & Beverages','Inflight Entertainment','Ground Service','Value For Money',  
  'Recommended']]

# rename dataframe
df.columns = ['airline_name', 'author', 'date', 'content', 'type_traveller','cabin_flown', 'overall_rating', 'seat_comfort_rating',
'cabin_staff_rating', 'food_beverages_rating','inflight_entertainment_rating', 'ground_service_rating','value_money_rating', 'recommended']


# Save to CSV
df.to_csv('Data/jetstar_airways.csv',index=False)

In [3]:
# Sanity check
df.head()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
0,jetstar airways,Saul Geffen,2020-05-06,Not Verified | Brisbane to Melbourne. Just li...,Solo Leisure,Economy Class,1,1,1,NaN,NaN,1,1,no
1,jetstar airways,E Marcerios,2020-05-05,✅ Trip Verified | My friend and I were travel...,Couple Leisure,Economy Class,1,NaN,NaN,NaN,NaN,NaN,1,no
2,jetstar airways,Chris Pokrzywa,2020-04-27,✅ Trip Verified | I'm absolutely appalled Jets...,Couple Leisure,Business Class,1,1,1,NaN,NaN,1,1,no
3,jetstar airways,Caryn Jervis,2020-04-21,✅ Trip Verified | Extremely poor customer serv...,Solo Leisure,Economy Class,1,1,1,NaN,NaN,1,1,no
4,jetstar airways,A Gurten,2020-04-15,✅ Trip Verified | I am very disappointed with...,Family Leisure,Economy Class,1,NaN,NaN,NaN,NaN,1,1,no


In [4]:
df.tail()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
95,jetstar airways,Paul Simonides,2019-12-08,Not Verified | Adelaide to Sydney. I've been ...,Family Leisure,Economy Class,1,NaN,NaN,NaN,NaN,1,1,no
96,jetstar airways,Victor Hew,2019-12-07,✅ Trip Verified | Sydney to Melbourne. If the...,Solo Leisure,Economy Class,1,1,1,NaN,NaN,1,1,no
97,jetstar airways,Ian Fleming,2019-12-06,✅ Trip Verified | Newcastle to Melbourne. Boo...,Couple Leisure,Economy Class,2,2,3,NaN,NaN,4,3,no
98,jetstar airways,J Ryd,2019-12-03,Not Verified | Queenstown to Sydney. My fligh...,Solo Leisure,Economy Class,1,1,2,1,1,1,1,no
99,jetstar airways,D Varunic,2019-12-02,✅ Trip Verified | Sunshine Coast to Sydney. T...,Business,Economy Class,1,1,1,1,1,1,1,no
